<a href="https://colab.research.google.com/github/juapgaua/ProtejAI/blob/main/ProjetoProtejAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai PyPDF2

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from PyPDF2 import PdfReader
import textwrap
from IPython.display import display
from IPython.display import Markdown

In [ ]:
# Configuração da API
try:
  api_key = userdata.get('SECRET_KEY')
  genai.configure(api_key=api_key)
except Exception as e:
  print(f"Erro ao configurar a API: {e}")
  exit()


In [ ]:
# Carregando o modelo Gemini
model = genai.GenerativeModel('gemini-pro')

In [ ]:
# Função para formatar a saída como Markdown
def to_markdown(text):
    text = text.replace('.', '.\n')
    return Markdown(textwrap.indent(text, '>', predicate=lambda x: True))

In [ ]:
# Função para extrair texto de um arquivo PDF
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:
# Função para avaliar um aspecto da política de privacidade
def evaluate_aspect(aspect, policy_text):
    prompt = f"""
    Você é um especialista em LGPD.
    Com base na política de privacidade fornecida, avalie o seguinte aspecto em relação à conformidade com a LGPD: {aspect}

    Política de Privacidade:
    {policy_text}

    Seja conciso e objetivo em sua análise.
    """
    response = model.generate_content(prompt)
    return response.text.strip()

In [ ]:
# Loop principal do chatbot
chat = model.start_chat(history=[])
print("Chatbot LGPD iniciado! Vamos analisar sua política de privacidade.")
print('-------------------------------------------------------------')

# Obtém a política de privacidade do usuário
input_method = input("Digite 'texto' para digitar a política ou 'arquivo' para anexar um PDF: ")
if input_method.lower() == 'texto':
    policy_text = input("Digite a política de privacidade: ")
elif input_method.lower() == 'arquivo':
    file_path = input("Digite o caminho do arquivo PDF: ")
    try:
        policy_text = extract_text_from_pdf(file_path)
    except Exception as e:
        print(f"Erro ao ler o arquivo PDF: {e}")
        exit()
else:
    print("Método de entrada inválido.")
    exit()

# Aspectos a serem avaliados
aspectos = [
    "Finalidade da coleta de dados",
    "Base legal para o tratamento de dados",
    "Direitos dos titulares dos dados",
    "Medidas de segurança",
    "Transferência internacional de dados"
]

# Avalia cada aspecto da política
for aspect in aspectos:
    print(f"**Avaliando: {aspect}**")
    analysis = evaluate_aspect(aspect, policy_text)
    print('Resposta:\n', analysis, '\n')
    print('-------------------------------------------------------------')